In [2]:
import zipfile
import os

# Define paths
raw_data_path = "data/raw/LBSM_All_London.zip"
extract_dir = "data/raw/"

# Unzip the file
with zipfile.ZipFile(raw_data_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Unzipped to: {extract_dir}")

Unzipped to: data/raw/


In [3]:
import pandas as pd

# Define the CSV path
csv_path = os.path.join(extract_dir, "LBSM_All_London.csv")

# Load data (adjust chunksize if memory issues)
try:
    df = pd.read_csv(csv_path)
    print("Data loaded successfully!")
except Exception as e:
    print(f"Error: {e}")

/var/folders/5n/l1swydn11bg6c9v7_60kdfl00000gn/T/ipykernel_2137/1478309273.py:8: DtypeWarning: Columns (22,23,26,28,32,34,36,38,40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


Data loaded successfully!


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4710943 entries, 0 to 4710942
Data columns (total 43 columns):
 #   Column                                                Dtype  
---  ------                                                -----  
 0   UPRN                                                  int64  
 1   SCU_ID                                                float64
 2   OS_TOPO_TOID                                          object 
 3   EASTING                                               float64
 4   NORTHING                                              float64
 5   LATITUDE                                              float64
 6   LONGITUDE                                             float64
 7   OA                                                    object 
 8   LSOA                                                  object 
 9   MSOA                                                  object 
 10  OAC                                                   object 
 11  WARD_CODE  

In [7]:
df.describe()

,UPRN,SCU_ID,EASTING,NORTHING,LATITUDE,LONGITUDE,ESTIMATED_FLOOR_COUNT,ESTIMATE_TOTAL_FLOOR_AREA_ALL,SCU_FOOTPRINT,MEAN_OBJECT_HEIGHT_M,...,AGG_DEC_CURRENT_OPERATIONAL_RATING_BY_COUNT,AGG_DEC_CURRENT_OPERATIONAL_RATING_BY_FLOORAREA,EPC_RATING_COUNT_DOM,EPC_RATING_COUNT_NONDOM,AGG_EPC_CURRENT_ENERGY_EFFICIENCY_BY_COUNT,AGG_EPC_CURRENT_ENERGY_EFFICIENCY_BY_FLOORAREA,AGG_EPC_POTENTIAL_ENERGY_EFFICIENCY_BY_COUNT,AGG_EPC_POTENTIAL_ENERGY_EFFICIENCY_BY_FLOORAREA,AGG_RVEPC_CURRENT_ENERGY_EFFICIENCY_BY_COUNT,AGG_RVEPC_CURRENT_ENERGY_EFFICIENCY_BY_FLOORAREA
count,4.710943e+06,4.199335e+06,4.710943e+06,4.710943e+06,4.710943e+06,4.710943e+06,4.040839e+06,4.077612e+06,4.077612e+06,4.066068e+06,...,17708.000000,17708.000000,4.710943e+06,4.710943e+06,2.530888e+06,2.517743e+06,2.448279e+06,2.434926e+06,4.008957e+06,2.664385e+06
mean,4.869539e+10,5.524026e+13,5.305379e+05,1.802806e+05,5.150630e+01,-1.204738e-01,3.258466e+00,1.742062e+03,5.006609e+02,9.529609e+00,...,104.901288,106.800599,6.009054e+00,1.392012e-01,6.424351e+01,6.417145e+01,7.431736e+01,7.430454e+01,6.217071e+01,6.339902e+01
std,5.709857e+10,2.848321e+12,1.016541e+04,7.928817e+03,7.118639e-02,1.465662e-01,2.740113e+00,1.351462e+04,4.799278e+03,7.243100e+00,...,44.893911,44.281151,1.882490e+01,9.878525e-01,1.391911e+01,1.389497e+01,9.917972e+00,9.966806e+00,1.196736e+01,1.428779e+01
min,5.000001e+06,5.030100e+13,5.035990e+05,1.559100e+05,5.128719e+01,-5.098838e-01,0.000000e+00,0.000000e+00,1.350000e+00,0.000000e+00,...,0.000000,0.000000,0.000000e+00,0.000000e+00,-1.039000e+03,-1.039000e+03,-3.090000e+02,-3.090000e+02,-6.490000e+02,-1.039000e+03
25%,2.020345e+08,5.270110e+13,5.243550e+05,1.751410e+05,5.146018e+01,-2.098480e-01,2.000000e+00,1.050000e+02,6.163000e+01,5.920000e+00,...,76.000000,80.000000,0.000000e+00,0.000000e+00,5.800000e+01,5.800000e+01,7.100000e+01,7.100000e+01,5.600000e+01,5.700000e+01
50%,1.007027e+10,5.510120e+13,5.308160e+05,1.808950e+05,5.151243e+01,-1.165239e-01,2.000000e+00,1.790000e+02,9.412000e+01,7.240000e+00,...,95.000000,98.000000,1.000000e+00,0.000000e+00,6.600000e+01,6.600000e+01,7.600000e+01,7.600000e+01,6.300000e+01,6.500000e+01
75%,1.000215e+11,5.780105e+13,5.369910e+05,1.858690e+05,5.155623e+01,-2.714640e-02,4.000000e+00,8.699300e+02,3.208500e+02,1.081000e+01,...,121.000000,121.000000,3.000000e+00,0.000000e+00,7.200000e+01,7.200000e+01,8.000000e+01,8.000000e+01,6.900000e+01,7.200000e+01
max,2.000044e+11,5.990126e+13,5.616086e+05,2.008590e+05,5.169117e+01,3.290815e-01,1.870000e+02,1.211826e+06,5.385512e+05,5.188300e+02,...,1606.000000,1606.000000,3.900000e+02,6.400000e+01,2.240000e+02,2.230000e+02,2.270000e+02,2.260000e+02,2.240000e+02,2.230000e+02


In [5]:
df.isnull().sum()

UPRN                                                          0
SCU_ID                                                   511608
OS_TOPO_TOID                                                  0
EASTING                                                       0
NORTHING                                                      0
LATITUDE                                                      0
LONGITUDE                                                     0
OA                                                         4558
LSOA                                                       4558
MSOA                                                       4558
OAC                                                        4558
WARD_CODE                                                  4558
WARD                                                       4558
ADMINISTRATIVE_AREA                                           0
ESTIMATED_FLOOR_COUNT                                    670104
ESTIMATE_TOTAL_FLOOR_AREA_ALL           

In [8]:
(df.isnull().sum() / len(df)) * 100

UPRN                                                     0.000000
SCU_ID                                                  10.859991
OS_TOPO_TOID                                             0.000000
EASTING                                                  0.000000
NORTHING                                                 0.000000
LATITUDE                                                 0.000000
LONGITUDE                                                0.000000
OA                                                       0.096753
LSOA                                                     0.096753
MSOA                                                     0.096753
OAC                                                      0.096753
WARD_CODE                                                0.096753
WARD                                                     0.096753
ADMINISTRATIVE_AREA                                      0.000000
ESTIMATED_FLOOR_COUNT                                   14.224413
ESTIMATE_T

In [11]:
df = df.drop_duplicates()
high_missing_cols = [
    'BASEMENT_FLOOR',  # 93.1% missing
    'AGG_DEC_CURRENT_OPERATIONAL_RATING_BY_COUNT',  # 99.6% missing
    'AGG_DEC_CURRENT_OPERATIONAL_RATING_BAND_BY_COUNT',
    'AGG_DEC_CURRENT_OPERATIONAL_RATING_BY_FLOORAREA',
    'AGG_DEC_CURRENT_OPERATIONAL_RATING_BAND_BY_FLOORAREA',
    'DOMINANT_ND_ACTIVITY_BY_C2_FS',  # 85.6% missing
    'DOMINANT_ND_ACTIVITY_BY_C2_COUNT'  # 84.7% missing
]
df = df.drop(columns=high_missing_cols)

In [12]:
# Impute numeric columns with median
numeric_cols = ['ESTIMATED_FLOOR_COUNT', 'ESTIMATE_TOTAL_FLOOR_AREA_ALL', 'MEAN_OBJECT_HEIGHT_M']
for col in numeric_cols:
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)

# For EPC data: Drop missing rows (if critical) OR impute
# Option 1: Drop rows (strict but accurate)
df = df.dropna(subset=['AGG_EPC_CURRENT_ENERGY_EFFICIENCY_BY_COUNT'])

# Option 2: Impute with median (if retaining data volume is crucial)
# epc_median = df['AGG_EPC_CURRENT_ENERGY_EFFICIENCY_BY_COUNT'].median()
# df['AGG_EPC_CURRENT_ENERGY_EFFICIENCY_BY_COUNT'] = df['AGG_EPC_CURRENT_ENERGY_EFFICIENCY_BY_COUNT'].fillna(epc_median)